## Advent of code 2023 day 11-20
See https://adventofcode.com/

In [ ]:
# note that this notebook requires the .venv environment (which is set up with pypy3.10-v7.3.13-win64)
# to activate it from a git bash shell: source .venv/Scripts/activate

import collections
import itertools
import functools
import re
import copy
import math
import sys
import time
import json
import heapq
import bisect
import random
import dataclasses

import sortedcontainers

import zio

In [ ]:
# version check and timestamp
# NB the timestamp supports ranking using an honor system, before starting include this line
# in the header of your solution (which should start with a line like # 2019 day 2), then whenever you want save
# a private leaderboard json file, and run python privaterank.py filename.json

print(f'python version: {sys.version}')
print(f'# start_ts={int(time.time())}')

In [ ]:
# 2023 day 20 part 1
# mv ~/Downloads/input* data_src/2023-day-20-input.txt
# big input file looks like: modest list of connections
# idea: part 1 parse as a dict of key to type and a dict of key to destinations, then 
#  just simulate

sample2='''
broadcaster -> a, b, c
%a -> b
%b -> c
%c -> inv
&inv -> a
'''

sample3='''
broadcaster -> a
%a -> inv, con
&inv -> b
%b -> con
&con -> output
'''

def parse_input(lines):
    '''parse input into types and network/destinations'''
    mtypes={}
    mnet={}
    for s in lines:
        tup=s.replace(' ', '').split('->')
        assert len(tup)==2
        key=tup[0]
        dests=tup[1].split(',')
        if key.startswith('%'):
            key=key[1:]
            mtypes[key]='%'
        elif key.startswith('&'):
            key=key[1:]
            mtypes[key]='&'
        elif key=='broadcaster':
            mtypes[key]='b'
        mnet[key]=dests
    return mtypes, mnet

track_keys={'hc', 'qt', 'ck', 'kb'}
simtrack={} # maps tracked keys to list of button presses where they send out a low signal

def simulate_button(ntimes, mtypes, mnet):
    '''simulate pressing the button ntimes, returns score'''
    global track_keys, simtrack
    simtrack.clear()
    status=init_state(mtypes, mnet) # see init_state
    intodo=[] # list of (src, dest, is-hi-bool) message tuples being processed
    outtodo=[] # ditto, new messages being gathered
    lowcount=0
    hicount=0
    for buttoni in range(ntimes):
        intodo.append( (None, 'broadcaster', False) ) # False is low, True is hi
        while True: # loop over intodo, moving it into outtodo first
            if len(intodo)<1:
                break
            outtodo.extend(intodo)
            intodo.clear()
            for msgtup in outtodo:
                if msgtup[2]==False:
                    lowcount+=1
                else:
                    hicount+=1
                if msgtup[0] in track_keys and msgtup[2]==False:
                    tl=simtrack.setdefault(msgtup[0], [])
                    if len(tl)<1 or tl[-1]!=buttoni+1:
                        tl.append(buttoni+1)
                process_msg(msgtup[0], msgtup[1], msgtup[2], status, intodo, mtypes, mnet)
            outtodo.clear()
    return lowcount*hicount

def init_state(mtypes, mnet):
    '''create initial state based on specified data'''
    res={}
    for key,mt in mtypes.items():
        if mt=='%': # flipflop defaults to False=off
            res[key]=False
        elif mt=='&': # conjunction defaults to False for each input
            inps={}
            for key2, dests in mnet.items():
                if key in dests:
                    inps[key2]=False
            res[key]=inps
    return res

def process_msg(sname, dname, mishi, status, intodo, mtypes, mnet):
    '''process a single message being sent from module sname to module dname of type mishi (True is high)'''
    mtype=mtypes.get(dname)
    if mtype=='b': # broadcaster, no state
        for dest in mnet[dname]:
            intodo.append( (dname, dest, mishi) )
    elif mtype=='%': # flipflop, state is a boolean, False=off
        if not mishi: # only reacts to low signal
            newstate=not status[dname]
            status[dname]=newstate
            for dest in mnet[dname]:
                intodo.append( (dname, dest, newstate) )
    elif mtype=='&': # conjunction, state is a boolean per input, False=low
        inps=status[dname]
        assert sname in inps
        inps[sname]=mishi
        sig=not all(inps.values())
        for dest in mnet[dname]:
            intodo.append( (dname, dest, sig) )

sample1=open('data_src/2023-day-20-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]

# part 1
mtypes,mnet=parse_input(lines)
score=simulate_button(1000000, mtypes, mnet)
print(f'part 1: {score=}')

# part 1: 788848550
# NB pressing the button 100000000 times (100M) takes 6 minutes to process with result: 
#  lowcount=1722116149, hicount=4615037952, lowrxcount=0, hirxcount=547493440

In [ ]:
# 2023 day 20 part 2
# idea: monitoring the state of the conjunctions, there's a dependency tree of conjunctions of 
# which odd levels have to be true at the same time

# used manual analysis on the sample1 input file to determine hc, qt, ck and kb would have
# to send out low signals at the same time, added tracking for them in the part 1 code,
# below checked out their periodicity and simply calculated the least common multiplier of their periods

for k, v in simtrack.items():
    print(k, len(v), v[-10:])
    diffs=[]
    for i in range(len(v)-10, len(v)):
        diffs.append(v[i]-v[i-1])
    print('       ', diffs)
math.lcm(3793, 3911, 3917, 3929)

# part 2: 228300182686739

In [ ]:
# 2023 day 19 part 1
# start_ts=1702995791
# mv ~/Downloads/input* data_src/2023-day-19-input.txt
# big input file looks like: list of instructions and records
# idea: part 1 parse as a dict of lists of instructions and list of records, then just execute

sample2='''
px{a<2006:qkq,m>2090:A,rfg}
pv{a>1716:R,A}
lnx{m>1548:A,A}
rfg{s<537:gd,x>2440:R,A}
qs{s>3448:A,lnx}
qkq{x<1416:A,crn}
crn{x>2662:A,R}
in{s<1351:px,qqz}
qqz{s>2770:qs,m<1801:hdj,R}
gd{a>3333:R,R}
hdj{m>838:A,pv}

{x=787,m=2655,a=1222,s=2876}
{x=1679,m=44,a=2067,s=496}
{x=2036,m=264,a=79,s=2244}
{x=2461,m=1339,a=466,s=291}
{x=2127,m=1623,a=2188,s=1013}
'''

def parse_input(groups):
    '''parse into workflows and records'''
    assert len(groups)==2
    wfc={}
    for line in groups[0]:
        line=line.replace('{', ' ').replace('}', '')
        tup=line.split()
        assert len(tup)==2
        key=tup[0]
        assert key not in wfc
        tup=tup[1].split(',')
        rules=[]
        for rt in tup:
            if rt:
                rl=[ s for s in re.split(f'(\w+)', rt) if s and s!=':' ]
                rl=[ int(s) if s.isnumeric() else s for s in rl ] # would not support negative numbers, but data doesn't have those
                assert len(rl) in {1,4}
                rules.append( tuple(rl) )
        wfc[key]=rules
    data=[]
    for line in groups[1]:
        line=line.replace('{', '').replace('}', '')
        tup=line.split(',')
        assert len(tup)==4
        rec={}
        for t in tup:
            assert t[1]=='='
            rec[t[0]]=int(t[2:])
        data.append(rec)
    return wfc, data

def is_accepted(wfc, part):
    '''for specified part follow the workflow, return True if accepted at the end'''
    node='in'
    while node!='A' and node!='R':
        wf=wfc[node]
        nextnode=None
        for rule in wf:
            if len(rule)==1:
                nextnode=rule[0]
                break
            else:
                assert len(rule)==4
                assert rule[1]=='<' or rule[1]=='>'
                if (rule[1]=='<' and part[rule[0]]<rule[2]) or \
                   (rule[1]=='>' and part[rule[0]]>rule[2]):
                    nextnode=rule[3]
                    break
        assert nextnode is not None # each workflow should end with single-element rule
        node=nextnode
    return node=='A'

sample1=open('data_src/2023-day-19-input.txt').read()
groups=zio.get_line_groups(sample1.splitlines(), nostrip=False)
wfc,data=parse_input(groups)

# part 1
score=0
for part in data:
    if is_accepted(wfc, part):
        score+=part['x']+part['m']+part['a']+part['s']
print(f'part 1: {score=}')

# part 1: 346230

In [ ]:
# 2023 day 19 part 2
# idea: instead of sending a part through the workflows, send a 'part-of-value-ranges' through,
# on each individual rule there will be a split; one subset of the ranges takes the branch, the other subset continues.
# there could be loops for value combinations that were not touched in the actual input data, this should be detected
# otherwise each part should only visit each workflow once, so should terminate relatively quickly

def count_accepted(wfc, part, node, rule_index, visited):
    '''for specified part-of-ranges follow the workflow, splitting recursively, 
    returning the accepted combinated if accepted or 0 if rejected'''
    if node=='R':
        return 0
    if node=='A':
        res=1
        for v in part.values():
            res*=(v[1]-v[0])+1
        return res
    rule=wfc[node][rule_index]
    res=0
    if len(rule)==1:
        nextnode=rule[0]
        if nextnode in visited:
            print(f'loop {visited=} to {nextnode=} avoided')
        if nextnode not in visited:
            visited.add(nextnode)
            res+=count_accepted(wfc, part, nextnode, 0, visited)
            visited.remove(nextnode)
    else:
        assert len(rule)==4
        key,oper,val,nextnode=rule
        n,m=part[key]
        assert oper=='<' or oper=='>'
        if oper=='<':
            if val<=n: # always false so always node+1
                res+=count_accepted(wfc, part, node, rule_index+1, visited)
            elif val<=m: # split into n - val-1 (true) and val - m (false)
                if nextnode in visited:
                    print(f'loop {visited=} to {nextnode=} avoided')
                if nextnode not in visited:
                    visited.add(nextnode)
                    part[key]=(n,val-1)
                    assert part[key][1]>=part[key][0]
                    res+=count_accepted(wfc, part, nextnode, 0, visited)
                    part[key]=(n,m) # restore
                    visited.remove(nextnode)
                part[key]=(val,m)
                assert part[key][1]>=part[key][0]
                res+=count_accepted(wfc, part, node, rule_index+1, visited)
                part[key]=(n,m) # restore
            else: # val>m, always true so always nextnode
                if nextnode in visited:
                    print(f'loop {visited=} to {nextnode=} avoided')
                if nextnode not in visited:
                    visited.add(nextnode)
                    res+=count_accepted(wfc, part, nextnode, 0, visited)
                    visited.remove(nextnode)
        else: # oper=='>'
            if val<n: # always true so always nextnode
                if nextnode in visited:
                    print(f'loop {visited=} to {nextnode=} avoided')
                if nextnode not in visited:
                    visited.add(nextnode)
                    res+=count_accepted(wfc, part, nextnode, 0, visited)
                    visited.remove(nextnode)
            elif val<m: # split into val+1 - m (true) and n - val (false)
                if nextnode in visited:
                    print(f'loop {visited=} to {nextnode=} avoided')
                if nextnode not in visited:
                    visited.add(nextnode)
                    part[key]=(val+1,m)
                    assert part[key][1]>=part[key][0]
                    res+=count_accepted(wfc, part, nextnode, 0, visited)
                    part[key]=(n,m) # restore
                    visited.remove(nextnode)
                part[key]=(n,val)
                assert part[key][1]>=part[key][0]
                res+=count_accepted(wfc, part, node, rule_index+1, visited)
                part[key]=(n,m) # restore
            else: # val>=m, always false so always node+1
                res+=count_accepted(wfc, part, node, rule_index+1, visited)
    return res

sample1=open('data_src/2023-day-19-input.txt').read()
groups=zio.get_line_groups(sample1.splitlines(), nostrip=False)
wfc,data=parse_input(groups)

# part 2
part=dict(x=(1,4000), m=(1,4000), a=(1,4000), s=(1,4000))
score=count_accepted(wfc, part, 'in', 0, {'in'})
print(f'part 2: {score=}')

# part 2: 124693661917133

In [ ]:
# 2023 day 18 part 1
# start_ts=1702875898
# mv ~/Downloads/input* data_src/2023-day-18-input.txt
# big input file looks like: 786 lines of codes
# idea: part 1 parse as instruction lines, then paint into dict board,
# then floodfill outside, counting the inside, for now ignoring color

sample2='''
R 6 (#70c710)
D 5 (#0dc571)
L 2 (#5713f0)
D 2 (#d2c081)
R 2 (#59c680)
D 2 (#411b91)
L 5 (#8ceee2)
U 2 (#caa173)
L 1 (#1b58a2)
U 2 (#caa171)
R 2 (#7807d2)
U 3 (#a77fa3)
L 2 (#015232)
U 2 (#7a21e3)
'''

def paint_board(lines):
    res={}
    x=0
    y=0
    res[ (x,y) ]=1
    for tup in lines:
        dir,n,clr=tup
        n=int(n)
        for _ in range(n):
            if dir=='R':
                x+=1
            elif dir=='D':
                y+=1
            elif dir=='L':
                x-=1
            else:
                assert dir=='U'
                y-=1
            res[ (x,y) ]=1
    return res

def paint_exterior(board):
    xlist=sorted({ xy[0] for xy in board })
    ylist=sorted({ xy[1] for xy in board })
    sx=xlist[0]-1
    sy=ylist[0]-1
    w=xlist[-1]-sx+3
    h=ylist[-1]-sy+3
    todo={ (sx,sy) }

    def paint_e_step(x,y):
        if x<sx or y<sy or x>=sx+w or y>=sy+h:
            return
        todo.add( (x,y) )

    while len(todo)>0:
        xy=todo.pop()
        if xy in board:
            continue
        board[xy]=2
        x,y=xy
        paint_e_step(x+1, y)
        paint_e_step(x-1, y)
        paint_e_step(x, y+1)
        paint_e_step(x, y-1)
    # now also paint interior
    for x in range(sx,sx+w):
        for y in range(sy,sy+h):
            if (x,y) not in board:
                board[ (x,y) ]=0
    return board

def count_non_ext(board):
    res=0
    for xy, val in board.items():
        if val==0 or val==1:
            res+=1
    return res

sample1=open('data_src/2023-day-18-input.txt').read()
lines=[s.split() for s in sample1.splitlines() if len(s)>0 ]

# part 1
board=paint_board(lines)
board2=paint_exterior(board)
score=count_non_ext(board2)
print(f'part 1: {score=}')

# part 1: 70026

In [ ]:
# 2023 day 18 part 2 implementation B
# grid-based - painting the board creates a grid consisting of the x and y of all vertices,
# can then iterate over each rectangle in the grid, each has four sides that have a drawn border or not,
# it has a drawn area that can hold lava anyway, and a non-drawn area that we have to check,
# now paint outside to determine what's inside, what's outside, now add all drawn and non-drawn-but-inside areas 
# together, NB the grid cells overlap at the edges, here we assign the border being closed or not to both adjacent cells,
# but the area of the right and bottom edges of each cell is not counted in the cell area

@dataclasses.dataclass
class Lineseg: # used in impl. A and B
    x1: int=None
    y1: int=None
    x2: int=None
    y2: int=None

def paint_board(instr): # used in impl. A and B
    '''converts drawing instructions into a list of line segments, with ordered coords'''
    res=[]
    sx=0
    sy=0
    x=sx
    y=sy
    for tup in instr:
        dir,n=tup
        assert n>0
        if dir=='R':
            x2=x+n
            y2=y
            res.append(Lineseg(x1=x, y1=y, x2=x2, y2=y2))
        elif dir=='D':
            x2=x
            y2=y+n
            res.append(Lineseg(x1=x, y1=y, x2=x2, y2=y2))
        elif dir=='L':
            x2=x-n
            y2=y
            res.append(Lineseg(x1=x2, y1=y2, x2=x, y2=y))
        else:
            assert dir=='U'
            x2=x
            y2=y-n
            res.append(Lineseg(x1=x2, y1=y2, x2=x, y2=y))
        x=x2
        y=y2
    assert x==sx # back at the start?!
    assert y==sy
    return res

def get_seg_len(seg): # used in impl. A and B
    '''segment length including both end points'''
    return abs(seg.x2-seg.x1)+abs(seg.y2-seg.y1)+1

def is_insegs(segs, x, y):
    '''is this coordinate painted/drawn over?'''
    for seg in segs:
        if seg.x1<=x<=seg.x2 and seg.y1<=y<=seg.y2:
            return True
    return False

@dataclasses.dataclass
class CellData:
    paintx: int=None # upper left
    painty: int=None # upper left
    paintw: int=None # width
    painth: int=None # height
    closed_L: bool=None # is cell side painted closed or not
    closed_R: bool=None
    closed_U: bool=None
    closed_D: bool=None
    drawn_area: int=None # number of painted cells into (along edge)
    nondrawn_area: int=None # number of non-painted cells inside

def fill_celldata(segs):
    gridx=sorted({ seg.x1 for seg in segs } | { seg.x2 for seg in segs })
    gridx.append(gridx[-1]+5) # add an empty & open column of cells to make drawing edges easier
    gridy=sorted({ seg.y1 for seg in segs } | { seg.y2 for seg in segs })
    gridy.append(gridy[-1]+5) # also add a row
    celldata={} # maps x,y of grid cell to CellData
    # fill cell data
    for xi in range(1, len(gridx)):
        for yi in range(1, len(gridy)):
            cd=CellData()
            celldata[ (xi-1, yi-1) ]=cd
            cd.paintx=gridx[xi-1]
            cd.paintw=gridx[xi]-gridx[xi-1]
            cd.painty=gridy[yi-1]
            cd.painth=gridy[yi]-gridy[yi-1]
            cd.closed_L=is_insegs(segs, gridx[xi-1], gridy[yi-1]+cd.painth//2)
            cd.closed_R=is_insegs(segs, gridx[xi], gridy[yi-1]+cd.painth//2)
            cd.closed_U=is_insegs(segs, gridx[xi-1]+cd.paintw//2, gridy[yi-1])
            cd.closed_D=is_insegs(segs, gridx[xi-1]+cd.paintw//2, gridy[yi])
            cd.drawn_area=0
            if cd.closed_L:
                cd.drawn_area+=cd.painth
            if cd.closed_U:
                cd.drawn_area+=cd.paintw
            if cd.closed_L and cd.closed_U:
                cd.drawn_area-=1 # corner cell counted double
            # even if this cell isn't closed left/upper we may have to paint the upper left pixel if the cell above
            # is closed left or the cell to our left is closed up
            if cd.closed_L==False and cd.closed_U==False and ((xi>1 and celldata[(xi-2, yi-1)].closed_U) or \
                                                              (yi>1 and celldata[(xi-1, yi-2)].closed_L)):
                cd.drawn_area+=1
            cd.nondrawn_area=cd.paintw*cd.painth-cd.drawn_area
    # check drawn area calculations
    totalseglen=sum([get_seg_len(seg)-1 for seg in segs]) # assumes it's a closed loop
    totalareadrawnlen=sum([cd.drawn_area for cd in celldata.values()])
    print(f'{totalseglen=}, {totalareadrawnlen=}')
    assert totalseglen==totalareadrawnlen
    return celldata

def paint_outside(celldata):
    xlist=sorted({ xy[0] for xy in celldata.keys() })
    ylist=sorted({ xy[1] for xy in celldata.keys() })
    sx=xlist[0]-1
    sy=ylist[0]-1
    w=xlist[-1]-sx+3
    h=ylist[-1]-sy+3
    todo={ (sx,sy) }
    outside=set() # x,y of cells outside drawn area

    def paint_e_step(x,y, checkflag):
        if x<sx or y<sy or x>=sx+w or y>=sy+h:
            return
        cd=celldata.get( (x,y) )
        if cd is not None:
            if checkflag=='L' and cd.closed_L:
                return
            if checkflag=='R' and cd.closed_R:
                return
            if checkflag=='U' and cd.closed_U:
                return
            if checkflag=='D' and cd.closed_D:
                return
        todo.add( (x,y) )

    while len(todo)>0:
        xy=todo.pop()
        if xy in outside:
            continue
        outside.add(xy)
        x,y=xy
        paint_e_step(x+1, y, 'L')
        paint_e_step(x-1, y, 'R')
        paint_e_step(x, y+1, 'U')
        paint_e_step(x, y-1, 'D')
    return outside

def count_areas(celldata, outside):
    area=0
    for xy, cd in celldata.items():
        area+=cd.drawn_area
        if xy in outside:
            continue
        area+=cd.nondrawn_area
    return area

def map_board(segs):
    '''convert line segments into a grid'''
    celldata=fill_celldata(segs)
    outside=paint_outside(celldata)
    area=count_areas(celldata, outside)
    return area

sample1=open('data_src/2023-day-18-input.txt').read()
lines=[s.split() for s in sample1.splitlines() if len(s)>0 ]
trueinstr=[]
for tup in lines:
    s=tup[2].strip()
    s=s.replace('(', '').replace(')', '').replace('#', '')
    dir={0: 'R', 1: 'D', 2: 'L', 3: 'U'}[int(s[-1])]
    n=int(s[:-1], base=16)
    trueinstr.append( (dir,n) )

segs=paint_board(trueinstr)
score=map_board(segs)
print(f'part 2: {score=}')

# part 2: 68548301037382

In [ ]:
# 2023 day 18 part 2 implementation A
# line segments - in the end too complicated and also incorrect
# (when counting one horizontal and two vertical segments, it's not clear 
# whether that area is inside the border)

def get_vertices(segs):
    data={} # maps (x,y) to set of hor and set of vert segment indices that touch there,
    # dots of length one in both dimensions are counted as horizontal

    def add_xy(x, y, hori, verti):
        dm=data.setdefault( (x,y), {})
        if len(dm)<2:
            dm['hor']=set()
            dm['vert']=set()
        if hori is not None:
            dm['hor'].add(hori)
        if verti is not None:
            dm['vert'].add(verti)

    for segi, seg in enumerate(segs):
        if seg.y1==seg.y2: # hor or dot
            add_xy(seg.x1, seg.y1, segi, None)
            add_xy(seg.x2, seg.y2, segi, None)
        else:
            assert seg.x1==seg.x2 # vert
            add_xy(seg.x1, seg.y1, None, segi)
            add_xy(seg.x2, seg.y2, None, segi)
    return data

def check_segs(segs):
    '''check that every vertex has two connected lines, one horizontal, one vertical,
    and there are no dots'''
    for seg in segs:
        if get_seg_len(seg)<=1:
            print(f'check_segs failed; dot: {seg.__dict__}')
            return False
    data=get_vertices(segs)
    for xy, dm in data.items():
        if len(dm['hor'])!=1:
            print(f'check_segs failed; {len(dm["hor"])} horizontals touching at {xy}')
            return False
        if len(dm['vert'])!=1:
            print(f'check_segs failed; {len(dm["vert"])} horizontals touching at {xy}')
            return False
    return True

def find_seg_highhor(segs):
    '''find highest horizontal seg, return index'''
    miny=None
    besti=None
    for i,seg in enumerate(segs):
        if seg.y1!=seg.y2:
            continue
        if miny is None or seg.y1<miny:
            miny=seg.y1
            besti=i
    return besti

def find_seg_vert_connect(segs, x, y, nocheckdownward=False):
    '''find vertical seg connecting to x,y, return index, check there is only 1'''
    besti=None
    for i,seg in enumerate(segs):
        if seg.x1!=seg.x2:
            continue
        assert seg.y1<=seg.y2
        if not nocheckdownward:
            assert y<=seg.y1
        if (seg.x1==x and seg.y1==y) or (seg.x2==x and seg.y2==y):
            assert besti is None # should be only one
            besti=i
    print(f'find_seg_vert_connect {x=}, {y=}: {besti=}')
    return besti

def get_segs_touching(seg1, seg2):
    '''do these two share an (end) vertex?'''
    return (seg1.x1==seg2.x1 and seg1.y1==seg2.y1) or (seg1.x1==seg2.x2 and seg1.y1==seg2.y2) or \
        (seg1.x2==seg2.x1 and seg1.y2==seg2.y1) or (seg1.x2==seg2.x2 and seg1.y2==seg2.y2)

def remove_segs(segs, segil):
    '''remove segments by index (while likely mess up indices you're still holding)'''
    for i in reversed(sorted(segil)):
        segs.pop(i)

def split_segvert(segs, vseg, toplen):
    '''split a vertical segment into a piece of toplen and remainder'''
    assert vseg.x1==vseg.x2
    assert get_seg_len(vseg)>toplen
    newseg=Lineseg(x1=vseg.x1, y1=vseg.y1+toplen-1, x2=vseg.x1, y2=vseg.y2)
    vseg.y2=vseg.y1+toplen-1
    segs.append(newseg)

def find_2_hor(segs):
    '''find and return two hortizontal segments that touch'''
    data=get_vertices(segs)
    for xy, dm in data.items():
        if len(dm['hor'])>1:
            assert len(dm['hor'])==2
            return tuple(dm['hor'])
    return None

def join_hors(segs):
    '''join touching horizontal segments, NB they can also overlap, return any to be counted cells that
    were removed'''
    res=0
    while True:
        htup=find_2_hor(segs)
        if htup is None:
            break
        hor1i=htup[0]
        hor2i=htup[1]
        if segs[hor1i].x1>segs[hor2i].x1:
            dum=hor1i
            hor1i=hor2i
            hor2i=dum
        seg1=segs[hor1i]
        seg2=segs[hor2i]
        hory=seg1.y1
        assert hory==seg1.y2==seg2.y1==seg2.y2 # horizontal
        print(f'join_hors seg1: {seg1.__dict__}, seg2: {seg2.__dict__}')
        # remove old segs, make new ones as needed (where verts connect), 
        # count any extruding segments that are now chopped off
        xes=sorted({ seg1.x1, seg1.x2, seg2.x1, seg2.x2 })
        print(f'join_hors {xes=}')
        remove_segs(segs, [hor1i, hor2i])
        if len(xes)==2: # two exactly overlapping segments, one left
            newxpairs=[(xes[0], xes[1])]
        elif len(xes)==3: # two adjoining segments, or only one if not connected in the middle
            verti=find_seg_vert_connect(segs, xes[1], hory, nocheckdownward=True)
            if verti is None:
                newxpairs=[(xes[0], xes[2])]
            else:
                newxpairs=[(xes[0], xes[1]), (xes[1], xes[2])]
        else:
            assert False # cannot be left with a dot(?) or two non-touching segments
        nremoved=0 # how many segments are chopped off?
        for xpair in newxpairs:
            vertti1=find_seg_vert_connect(segs, xpair[0], hory, nocheckdownward=True)
            vertti2=find_seg_vert_connect(segs, xpair[1], hory, nocheckdownward=True)
            if (vertti1 is not None) and (vertti2 is not None): # touched on both sides, keep
                newseg=Lineseg(x1=xpair[0], y1=hory, x2=xpair[1], y2=hory)
                segs.append(newseg)
            else: # not touched on both sides, chop off
                res+=xpair[1]-xpair[0]+1
                nremoved+=1
        if nremoved==2:
            res-=1 # one cell counted double
        print(f'join_hors {newxpairs=}, {nremoved=}')
    return res

def count_area(segs):
    '''count area bounded by the line segments, knowing they form a closed boundary of rectangles'''
    # find one of the highest horizontal segs, and its two connecting vert segs, split longest into two,
    # create a new seg connecting the two bottom points, calculate piece (rectangle) you've cut off,
    # remove the 3 segs from the list (always add new segments to the end to maintain seg indices)
    res=0
    while len(segs)>0:
        hori=find_seg_highhor(segs)
        assert hori is not None
        vert1i=find_seg_vert_connect(segs, segs[hori].x1, segs[hori].y1)
        if vert1i is None: # just a bottom line, calc its length and remove
            print('vert1i not found')
            len1=get_seg_len(segs[hori])
            res+=len1
            remove_segs(segs, [hori, ])
            continue
        vert2i=find_seg_vert_connect(segs, segs[hori].x2, segs[hori].y2)
        if vert2i is None:
            print('vert2i not found')
            assert get_seg_len(segs[vert1i])==1 # vert1i is just a 1x1 dot
            len1=get_seg_len(segs[hori])
            res+=len1
            remove_segs(segs, [hori, vert1i])
            continue
        if segs[vert1i].x1>segs[vert2i].x1: # swap them to keep them left-to-right
            dum=vert1i
            vert1i=vert2i
            vert2i=dum
        print(f'nicely found hori={segs[hori].__dict__}, vert1={segs[vert1i].__dict__}, vert2={segs[vert2i].__dict__}')
        len1=get_seg_len(segs[vert1i])
        len2=get_seg_len(segs[vert2i])
        print(f'{len1=}, {len2=}')
        if len1>len2:
            split_segvert(segs, segs[vert1i], min(len1, len2))
            len1=get_seg_len(segs[vert1i])
        elif len2>len1:
            split_segvert(segs, segs[vert2i], min(len1, len2))
            len2=get_seg_len(segs[vert2i])
        assert len1==len2
        newhor=Lineseg(x1=segs[vert1i].x2, y1=segs[vert1i].y2, x2=segs[vert2i].x2, y2=segs[vert2i].y2)
        print(f'adding newhor {newhor.__dict__}')
        segs.append(newhor)
        res+=(len1-1)*get_seg_len(segs[hori])
        remove_segs(segs, [hori, vert1i, vert2i])
        res+=join_hors(segs)
        #if not check_segs(segs):
        #    print('remaining')
        #    for seg in segs:
        #        print(seg.__dict__)
        #    print('done')
        #    assert False
    return res

sample1=open('data_src/2023-day-18-input.txt').read()
lines=[s.split() for s in sample2.splitlines() if len(s)>0 ]
trueinstr=[]
for tup in lines:
    s=tup[2].strip()
    s=s.replace('(', '').replace(')', '').replace('#', '')
    dir={0: 'R', 1: 'D', 2: 'L', 3: 'U'}[int(s[-1])]
    n=int(s[:-1], base=16)
    trueinstr.append( (dir,n) )

segs=paint_board(trueinstr)
assert check_segs(segs)
score=count_area(segs)
print(f'part 2: {score=}')

In [ ]:
# 2023 day 17
# start_ts=1702796449
# mv ~/Downloads/input* data_src/2023-day-17-input.txt
# big input file looks like: a big map
# idea: part 1 parse as a text map, then BFS
# part 2: same, but with explicit minconseqmoves and maxconseqmoves parameters

sample2='''
2413432311323
3215453535623
3255245654254
3446585845452
4546657867536
1438598798454
4457876987766
3637877979653
4654967986887
4564679986453
1224686865563
2546548887735
4322674655533
'''

sample3='''
111111111111
999999999991
999999999991
999999999991
999999999991
'''

# per board cell maintain minimum heat loss up to then, and remaining direction count
# at most 3 blocks in a single direction: in todo keep last direction and count,
# no reversing direction: check last direction as well,
# direction: 0-moving-to-right, 1-down, 2-left, 3-up

def min_heat_loss(lines, sx, sy, ex, ey, minconseqmoves, maxconseqmoves):
    '''BFS search for minimum heat loss'''
    # inner functions and array-mapped data for speed, todo as two arrays for fairness and efficiency
    w=len(lines[0])
    h=len(lines)

    # heat loss when entering a cell
    cell_heatloss=[ None ] * (w*h)
    for y,row in enumerate(lines):
        assert len(row)==w
        for x,c in enumerate(row):
            assert int(c)>0
            cell_heatloss[w*y+x]=int(c)
    
    # minimum heat loss so far per combination of x,y,dir,remaining-direction-count-in-this-direction, mapped into array
    board=[ None ] * (w*h*4*maxconseqmoves)
    print(f'{w=}, {h=}, {maxconseqmoves=}, {len(board)=}')
    bestfinalhl=None # best heat loss so far in end cell
    intodo=[] 
    outtodo=[] # lists of new x,y and incoming direction and remaining direction count (excluding this step)
    # and heat loss (including this step), direction: 0-moving-to-right, 1-down, 2-left, 3-up
    # intodo for new entries, outtodo for processing them, avoids inpredictable order of using a set

    def set_board(x,y,dir,rdc, newval):
        '''updates a heatloss value in board'''
        i=(w*y+x)*4*maxconseqmoves+dir*maxconseqmoves+rdc
        board[i]=newval

    def get_board(x,y,dir,rdc):
        '''returns a heatloss value from board'''
        i=(w*y+x)*4*maxconseqmoves+dir*maxconseqmoves+rdc
        return board[i]
    
    def get_heatloss(x,y):
        '''returns a heatloss value for a specific cell'''
        return cell_heatloss[w*y+x]
    
    def add_todo(x, y, newdir, olddir, oldrdc, oldheatloss, newrdc=None):
        '''(try to) add a todo moving to x,y moving in direction newdir'''
        assert oldrdc>=0
        if olddir==newdir and oldrdc==0:
            return
        if olddir==(newdir+2)%4: # opposite direction
            return
        if x<0 or y<0 or x>=w or y>=h:
            return
        if newrdc is None:
            newrdc=oldrdc-1 if olddir==newdir else maxconseqmoves-1
        newheatloss=oldheatloss+get_heatloss(x,y)
        if bestfinalhl is not None and newheatloss>=bestfinalhl:
            return
        todotup=(x,y,newdir,newrdc,newheatloss)
        intodo.append(todotup)

    def add_minmove_todo(x, y, dir, heatloss):
        '''add a todo at minimum move distance in specified direction (assumes direction was changed),
        starting from x,y'''
        rdc=maxconseqmoves
        for _ in range(minconseqmoves):
            if dir==0:
                x+=1
            elif dir==1:
                y+=1
            elif dir==2:
                x-=1
            else:
                assert dir==3
                y-=1
            if x<0 or y<0 or x>=w or y>=h:
                return
            heatloss+=get_heatloss(x,y)
            rdc-=1
        if bestfinalhl is not None and heatloss>=bestfinalhl:
            return
        todotup=(x,y,dir,rdc,heatloss)
        intodo.append(todotup)

    add_minmove_todo(sx, sy, 0, 0)
    add_minmove_todo(sx, sy, 1, 0)
    add_minmove_todo(sx, sy, 2, 0)
    add_minmove_todo(sx, sy, 3, 0)

    turn=0
    while True: # loop over intodo, moving it into outtodo first
        if len(intodo)<1:
            break
        outtodo.extend(intodo)
        intodo.clear()
        for todotup in outtodo:
            x,y,dir,rdc,heatloss=todotup
            bestheatloss=get_board(x,y,dir,rdc)
            if bestheatloss is None or heatloss<bestheatloss:
                set_board(x,y,dir,rdc,heatloss)
                if x==ex and y==ey and (bestfinalhl is None or heatloss<bestfinalhl):
                    bestfinalhl=heatloss
                # generate possible next moves
                if dir==0:
                    nx=x+1
                    ny=y
                elif dir==1:
                    nx=x
                    ny=y+1
                elif dir==2:
                    nx=x-1
                    ny=y
                else:
                    assert dir==3
                    nx=x
                    ny=y-1
                add_todo(nx, ny, dir, dir, rdc, heatloss) # move ahead
                add_minmove_todo(x, y, (dir+1)%4, heatloss) # turn clockwise
                add_minmove_todo(x, y, (dir-1)%4, heatloss) # turn counterclockwise
            if turn%100000000==0:
                print(f'{turn=}, {len(intodo)=}, {len(outtodo)=}, {bestfinalhl=}')
            turn+=1
        outtodo.clear()

    return bestfinalhl

sample1=open('data_src/2023-day-17-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]

# part 1
score=min_heat_loss(lines, 0, 0, len(lines[0])-1, len(lines)-1, 1, 3)
print(f'part 1: {score=}')

# part 2
score=min_heat_loss(lines, 0, 0, len(lines[0])-1, len(lines)-1, 4, 10)
print(f'part 2: {score=}')

# part 1: score=694
# part 2: score=829 (after 2m30s of running time for both parts together)

In [ ]:
# 2023 day 16 part 1
# start_ts=1702736717
# mv ~/Downloads/input* data_src/2023-day-16-input.txt
# big input file looks like: a big map
# idea: part 1 parse as a text board, then use BFS, 
#  considering a tile can be entered from 4 sides

sample2='''
.|...\\....
|.-.\\.....
.....|-...
........|.
..........
.........\\
..../.\\\\..
.-.-/..|..
.|....-|.\\
..//.|....
'''

def en_step(x, y, dir, lines, board, todo):
    '''single step to a next cell'''
    if x<0 or y<0 or y>=len(lines) or x>=len(lines[y]): # going outside
        return
    cs=board.setdefault( (x,y), set())
    if dir in cs: # did that direction already
        return
    cs.add(dir)
    todo.add( (x,y,dir) )

def energize1(lines, x, y, dir):
    '''simulate light bouncing around, energizing up to 4 incoming sides of each cell'''
    board={ (x,y): {dir} } # maps (x,y) to a set of 0-moving-to-right, 1-down, 2-left, 3-up
    todo={ (x,y,dir) }
    while len(todo)>0:
        tup=todo.pop()
        x,y,dir=tup
        c=lines[y][x]
        if dir==0: # to-right
            if c=='.' or c=='-':
                en_step(x+1, y, 0, lines, board, todo)
            elif c=='/':
                en_step(x, y-1, 3, lines, board, todo)
            elif c=='\\':
                en_step(x, y+1, 1, lines, board, todo)
            elif c=='|':
                en_step(x, y+1, 1, lines, board, todo)
                en_step(x, y-1, 3, lines, board, todo)
            else:
                assert False
        elif dir==1: # to-down
            if c=='.' or c=='|':
                en_step(x, y+1, 1, lines, board, todo)
            elif c=='/':
                en_step(x-1, y, 2, lines, board, todo)
            elif c=='\\':
                en_step(x+1, y, 0, lines, board, todo)
            elif c=='-':
                en_step(x+1, y, 0, lines, board, todo)
                en_step(x-1, y, 2, lines, board, todo)
            else:
                assert False
        elif dir==2: # to-left
            if c=='.' or c=='-':
                en_step(x-1, y, 2, lines, board, todo)
            elif c=='/':
                en_step(x, y+1, 1, lines, board, todo)
            elif c=='\\':
                en_step(x, y-1, 3, lines, board, todo)
            elif c=='|':
                en_step(x, y+1, 1, lines, board, todo)
                en_step(x, y-1, 3, lines, board, todo)
            else:
                assert False
        else: # to-up
            assert dir==3
            if c=='.' or c=='|':
                en_step(x, y-1, 3, lines, board, todo)
            elif c=='/':
                en_step(x+1, y, 0, lines, board, todo)
            elif c=='\\':
                en_step(x-1, y, 2, lines, board, todo)
            elif c=='-':
                en_step(x+1, y, 0, lines, board, todo)
                en_step(x-1, y, 2, lines, board, todo)
            else:
                assert False
    return board

sample1=open('data_src/2023-day-16-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]

# part 1
board=energize1(lines, 0, 0, 0)
print(f'part 1: count={len(board)}')

# part 1: 7046

In [ ]:
# 2023 day 16 part 2
# idea: just try them all and keep the maximum

maxcount=0
for y in range(len(lines)):
    board=energize1(lines, 0, y, 0)
    maxcount=max(maxcount, len(board))
    board=energize1(lines, len(lines[y])-1, y, 2)
    maxcount=max(maxcount, len(board))
for x in range(len(lines[0])):
    board=energize1(lines, x, 0, 1)
    maxcount=max(maxcount, len(board))
    board=energize1(lines, x, len(lines)-1, 3)
    maxcount=max(maxcount, len(board))
print(f'part 2: {maxcount=}')

# part 2: 7313

In [ ]:
# 2023 day 15 part 1
# start_ts=1702721585
# mv ~/Downloads/input* data_src/2023-day-15-input.txt
# big input file looks like: a single line of more than 20000 chars
# idea: part 1 parse as split by commas, then just run the hash function

sample2='''
rn=1,cm-,qp=3,cm=2,qp-,pc=4,ot=9,ab=5,pc-,pc=6,ot=7
'''

def hash(s):
    res=0
    for c in s:
        res+=ord(c)
        res=(17*res) % 256
    return res

sample1=open('data_src/2023-day-15-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
assert len(lines)==1
lines=lines[0].split(',')

# part 1
score=sum([hash(s) for s in lines])
print(f'part 1: {score=}')

# part 1: 505427

In [ ]:
# 2023 day 15 part 2
# perform hashmap

boxes={} # maps box id to an (ordered!) dict that maps label to focal length
for s in lines:
    if s.endswith('-'):
        label=s[:-1]
        focal=None
        bi=hash(label)
        box=boxes.setdefault(bi, {})
        if label in box:
            del box[label]
    else:
        i=s.index('=')
        assert i>=0
        label=s[:i]
        focal=int(s[i+1:])
        bi=hash(label)
        box=boxes.setdefault(bi, {})
        box[label]=focal
    #print(f'{s=}, {label=}, {bi=}, {focal=}, {boxes=}')

# focus power
score=0
for bi, box in boxes.items():
    for li, label in enumerate(list(box.keys())):
        focal=box[label]
        score+= (bi+1) * (li+1) * focal
print(f'part 2: {score=}')

# part 2: 243747

In [ ]:
# 2023 day 14 part 1
# start_ts=1702594360
# mv ~/Downloads/input* data_src/2023-day-14-input.txt
# big input file looks like: single big map
# idea: part 1 parse as lines, then just simulate the rolling and load calc.

sample2='''
O....#....
O.OO#....#
.....##...
OO.#O....O
.O.....O#.
O.#..O.#.#
..O..#O..O
.......O..
#....###..
#OO..#....
'''

sample1=open('data_src/2023-day-14-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]

def move_cell(board, xsrc, ysrc, xdest, ydest):
    board[ydest]=board[ydest][:xdest]+board[ysrc][xsrc]+board[ydest][xdest+1:]
    board[ysrc]=board[ysrc][:xsrc]+'.'+board[ysrc][xsrc+1:]

def roll_north(board):
    '''roll all Os north as far as possible'''
    for x in range(len(board[0])):
        for ydest in range(len(board)):
            if board[ydest][x]=='.': # empty destination spot
                for ysrc in range(ydest+1, len(board)):
                    if board[ysrc][x]=='O': # move it
                        move_cell(board, x, ysrc, x, ydest)
                        break
                    elif board[ysrc][x]=='#': # skip it
                        ydest=ysrc
                        break

def total_load_north(board):
    '''calc noth load (which starts counting south)'''
    res=0
    for y,row in enumerate(board):
        for x,c in enumerate(row):
            if c=='O':
                res+=len(board)-y
    return res

board=lines
roll_north(board)
#print('rolled:')
#for line in board:
#    print(line)
score=total_load_north(board)
print(f'part 1: {score=}')

# part 1: 113078

In [ ]:
# 2023 day 14 part 2 implementation A
# idea: for simplicity keep approach from part 1, just introduce a clockwise rotation, do n cycles to start, eg 100,
# after that determine a set of static rocks and each cycle compare the board with the previous one,
# subtract from set of statics,
# for dynamics per location create list of cycles where there's a rock there
# ouch - overlapping cycles so have to track individual rocks through both rolls and rotations :-(

def count_rrocks(board):
    '''return number of (potentially) rolling rocks'''
    n=0
    for y,row in enumerate(board):
        for x,c in enumerate(row):
            if c=='O':
                n+=1
    return n

def rotate_gr(board):
    '''rotate board 90 degrees clockwise'''
    res=[]
    for x in range(len(board[0])):
        row=''
        for y in range(len(board)-1, -1, -1):
            row+=board[y][x]
        res.append(row)
    return res

def do_cycle(board):
    '''single cycle of rolling in 4 directions'''
    for _ in range(4):
        roll_north(board)
        board=rotate_gr(board)
    return board

def update_movers(oldboard, board, movers, statics, ci):
    '''update tracking of moving rocks, oldboard is prev. board, ci is current cycle index
    (1-based)'''
    for y,row in enumerate(board):
        for x,c in enumerate(row):
            if c!=oldboard[y][x]:
                statics.discard( (x,y) )
            if ((x,y) not in statics) and c=='O':
                cl=movers.setdefault( (x,y) , [])
                cl.append(ci)

def create_future_board(board, movers, statics, ci):
    '''based on tracking show the board of specified ci'''
    # clear all movers, paint in all statics
    for xy in statics:
        x,y=xy
        assert board[y][x]=='O'
    for xy in movers.keys():
        x,y=xy
        board[y]=board[y][:x]+'.'+board[y][x+1:]
    # paint in the right movers
    for xy,mci in movers.items():
        x,y=xy
        period=mci[-1]-mci[-2]
        assert period== mci[-2]-mci[-3]
        if (ci-mci[-1]) % period == 0:
            board[y]=board[y][:x]+'O'+board[y][x+1:]
    return board

sample1=open('data_src/2023-day-14-input.txt').read()
lines=[s for s in sample2.splitlines() if len(s)>0 ]
board=lines
print(f'rrocks={count_rrocks(board)}')

startcyc=100
assert startcyc>(len(board)+len(board[0]))*2
for _ in range(startcyc):
    board=do_cycle(board)
statics=set() # rolling rocks that stay in place
for y,row in enumerate(board):
    for x,c in enumerate(row):
        if c=='O':
            statics.add( (x,y) )
extracyc=startcyc
movers={} # maps location to list of cycle ids when there's a rock there
for ci in range(extracyc):
    oldboard=copy.deepcopy(board)
    board=do_cycle(board)
    update_movers(oldboard, board, movers, statics, startcyc+ci+1)
print(f'after sim rrocks={count_rrocks(board)}')
print(f'{statics=}')
print(f'{movers=}')
board=create_future_board(board, movers, statics, 1000000000)
print(f'future rrocks={count_rrocks(board)}')
print('future board:')
for line in board:
    print(line)
score=total_load_north(board)
print(f'part 2: {score=}')

In [ ]:
# 2023 day 14 part 2 implementation B
# track individual rocks through both rolls and rotations :-(
# (turned out that some of these rocks have such long cycles that it took 200000
# to get them clear ?!, taking 86 minutes of calculation time - obviously not
# the best way to solve this although it does give the right answer in the end,
# could speed up the simulation quite a bit by properly implementing rolling to all sides,
# instead of rotating, and by maintaining the needed data structures throughout the cycles,
# still, must be a better way :-) )

sample2='''
O....#....
O.OO#....#
.....##...
OO.#O....O
.O.....O#.
O.#..O.#.#
..O..#O..O
.......O..
#....###..
#OO..#....
'''

def parse_board_b(lines):
    '''convert board as text to:
    unmovs is set of (x,y) of cube-rocks, movs is a map of id (rid) to (x,y) of each round-rock '''
    unmovs=set()
    movs={}
    rid=0
    for y,row in enumerate(lines):
        for x,c in enumerate(row):
            if c=='#':
                unmovs.add( (x,y) )
            elif c=='O':
                movs[rid]= (x,y)
                rid+=1
    return unmovs,movs
    
def roll_north_b(unmovs, movs):
    '''roll all Os/movs north as far as possible'''
    occup=set() # occupied cells
    occup.update(unmovs)
    occup.update(movs.values())
    while True:
        some_moved=False
        for rid in movs.keys():
            xy=movs[rid]
            x,y=xy
            while y>0 and (x,y-1) not in occup:
                occup.remove( (x,y) )
                y-=1
                xy= (x,y)
                occup.add( xy )
                movs[rid]= xy
                some_moved=True
        if not some_moved:
            break

def rotate_b(unmovs, movs):
    '''rotate board (unmovs and movs) 90 degrees clockwise'''
    lastrow=max([ xy[1] for xy in (unmovs | set(movs.values())) ])
    newunmovs=set()
    for xy in unmovs:
        x,y=xy
        newx=lastrow-y
        newy=x
        newunmovs.add( (newx, newy) )
    newmovs={}
    for rid, xy in movs.items():
        x,y=xy
        newx=lastrow-y
        newy=x
        newmovs[rid]= (newx, newy)
    unmovs.clear()
    unmovs.update(newunmovs)
    movs.clear()
    movs.update(newmovs)

def update_tracks_b(movs, tracks, ci):
    '''update tracks for current ci; maps rid to a map of (x,y) to a list of ci'''
    for rid, xy in movs.items():
        tm=tracks.setdefault(rid, {})
        cl=tm.setdefault(xy, [])
        cl.append(ci)

def do_cycle_b(unmovs, movs, tracks, ci, mintrackci):
    '''single cycle of rolling in 4 directions'''
    for _ in range(4):
        roll_north_b(unmovs, movs)
        rotate_b(unmovs, movs)
    if ci>=mintrackci:
        update_tracks_b(movs, tracks, ci)

def print_board_b(unmovs, movs, title):
    print(title+':')
    lastcol=max([ xy[0] for xy in (unmovs | set(movs.values())) ])
    lastrow=max([ xy[1] for xy in (unmovs | set(movs.values())) ])
    for y in range(lastrow+1):
        row=''
        for x in range(lastcol+1):
            if (x,y) in unmovs:
                row+='#'
            elif (x,y) in movs.values():
                row+='O'
            else:
                row+='.'
        print(row)
    print()

@dataclasses.dataclass
class XYItem:
    '''helper class to calculate the repeating period for a rock'''
    diffs=[] # list of repeating periods per position
    di: int=None # index into diffs
    total: int=0 # sum(diffs[di:])
    nexttotal: int=0 # sum(diffs[di-1:])

def cycle_future_board_b(unmovs, movs, tracks, target_ci):
    '''based on tracking create the board of specified target_ci'''
    movs.clear()
    # analyze tracks, based on that put the movs in the right spot on ci one by one
    for rid, tm in tracks.items():
        xydata=[] # list of XYItem
        cllenctr=collections.Counter()
        for cl in tm.values():
            cllenctr[len(cl)]+=1
        print(f'{rid=}, tracklengths: {cllenctr}')
        for xy, cl in tm.items():
            if len(cl)==1: # 'startup' position that was only visited once and can be ignored
                continue
            assert len(cl)>=6
            diffs=[]
            for i in range(1, len(cl)):
                diffs.append(cl[i]-cl[i-1])
            item=XYItem()
            item.diffs=diffs
            item.di=len(diffs)-1
            item.total=diffs[item.di]
            xydata.append(item)
            #print(f'{rid=}, {xy=}: starting on ci={cl[0]}, {diffs=}')
        # find least common cycle length for all positions through which this rock cycles
        while True:
            xydata.sort(key=lambda xyd: xyd.total)
            if xydata[0].total==xydata[-1].total: # all have the same value for total?
                break
            for xyd in xydata:
                xyd.nexttotal=xyd.total+xyd.diffs[xyd.di-1]
            xydata.sort(key=lambda xyd: xyd.nexttotal)
            xyd=xydata[0] # the first element has the lowest nexttotal, so advance that one
            xyd.total=xyd.nexttotal
            xyd.di-=1
        cyclen=xydata[0].total
        # look at last cyclen positions mod cyclen, should be only one matching
        locatci=[]
        minci=max([cl[-1] for cl in tm.values()])-cyclen+1
        for xy, cl in tm.items():
            for ci0 in reversed(cl):
                if ci0<minci:
                    break
                if ci0 % cyclen == target_ci%cyclen:
                    locatci.append(xy)
        print(f'{rid=}, {cyclen=}, {locatci=}')
        assert len(locatci)==1
        movs[rid]=locatci[0]
    # check no overlapping positions
    locctr=collections.Counter()
    for xy in movs.values():
        locctr[xy]+=1
    locoverlaps={ xy for xy,ct in locctr.items() if ct>1 }
    if len(locoverlaps)>0:
        print(f'{locoverlaps=}')
        assert False

def total_load_north_b(movs):
    '''calc north load (which starts counting south)'''
    lastrow=max([ xy[1] for xy in set(movs.values()) ])
    res=0
    for xy in movs.values():
        x,y=xy
        res+=lastrow+1-y
    return res

def total_load_north(board):
    res=0
    for y,row in enumerate(board):
        for x,c in enumerate(row):
            if c=='O':
                res+=len(board)-y
    return res

sample1=open('data_src/2023-day-14-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
unmovs,movs=parse_board_b(lines)
# unmovs is set of (x,y) of cube-rocks, movs is a map of id (rid) to (x,y) of each round-rock
print(f'rrocks={len(movs)}')
tracks={} # maps rid to a map of (x,y) to a list of ci

startcyc=200000
skipcyc=(len(lines)+len(lines[0]))*4
assert startcyc>=skipcyc*2
print(f'{startcyc=}, {len(lines)+len(lines[0])=}, {skipcyc=}')
for ci in range(startcyc):
    do_cycle_b(unmovs, movs, tracks, ci+1, skipcyc)
print(f'rrocks={len(movs)}')
#print(f'{tracks=}')
cycle_future_board_b(unmovs, movs, tracks, 1000000000)
score=total_load_north_b(movs)
print(f'part 2: {score=}')

# part 2: 94255 (after 86 minutes!)

In [ ]:
# 2023 day 13
# start_ts=1702590185
# mv ~/Downloads/input* data_src/2023-day-13-input.txt
# big input file looks like: bunch of maps
# idea: part 1 parse as groups of lines, then just iterate over possible reflection lines
# (instead of having horizontal and vertical reflection checks we use a rotate)

sample2='''
#.##..##.
..#.##.#.
##......#
##......#
..#.##.#.
..##..##.
#.#.##.#.

#...##..#
#....#..#
..##..###
#####.##.
#####.##.
..##..###
#....#..#
'''

sample1=open('data_src/2023-day-13-input.txt').read()
groups=zio.get_line_groups(sample1.splitlines(), nostrip=False)
print(f'ngroups={len(groups)}')

def find_reflect(board):
    '''find reflections across horizontal line, return lines above'''
    res=set()
    for n in range(1, len(board)): # try with n lines above fold
        failed=False
        i=0
        while n-1-i >= 0 and n+i < len(board):
            if board[n-1-i] != board[n+i]:
                failed=True
                break
            i+=1
        if not failed:
            res.add(n)
    return res

def rotate_gr(board):
    '''rotate board 90 degrees clockwise'''
    res=[]
    for x in range(len(board[0])):
        row=''
        for y in range(len(board)-1, -1, -1):
            row+=board[y][x]
        res.append(row)
    return res

# part 1
score=0
for groupi, group in enumerate(groups):
    lines=find_reflect(group)
    #print(f'{groupi=}: {lines=}')
    for n in lines:
        score+=100*n
    rgroup=rotate_gr(group)
    lines=find_reflect(rgroup)
    #print(f'r{groupi=}: {lines=}')
    for n in lines:
        score+=n
print(f'part 1: {score=}')

# part 1: 30802

# part 2
# just try all smudges and discard original lines, easy using sets
score=0
for groupi, group in enumerate(groups):
    lines=find_reflect(group)
    lines2=set()
    for y,row in enumerate(group):
        for x,c in enumerate(row): # try every smudge
            group[y]= group[y][:x]+('.' if c=='#' else '#')+group[y][x+1:]
            lines2.update(find_reflect(group))
            group[y]= group[y][:x]+c+group[y][x+1:]
    #print(f'{groupi=}: {lines=}, {lines2=}')
    lines=lines2-lines
    for n in lines:
        score+=100*n
    group=rotate_gr(group)
    lines=find_reflect(group)
    lines2=set()
    for y,row in enumerate(group):
        for x,c in enumerate(row): # try every smudge
            group[y]= group[y][:x]+('.' if c=='#' else '#')+group[y][x+1:]
            lines2.update(find_reflect(group))
            group[y]= group[y][:x]+c+group[y][x+1:]
    #print(f'r{groupi=}: {lines=}, {lines2=}')
    lines=lines2-lines
    for n in lines:
        score+=n
print(f'part 2: {score=}')

# part 2: 37876

In [ ]:
# 2023 day 12 part 0

def check_nums(s, nums, check_all):
    #print(f'check_nums: {s=}, {nums=}, {check_all=}')
    n=0
    in_dmg=False
    nums_idx=0
    for c in s:
        if in_dmg and c=='#':
            n+=1
        elif (not in_dmg) and c=='.':
            n=0
        elif c=='#':
            n=1
            in_dmg=True
        else:
            assert c=='.'
            if n>0:
                if nums_idx>=len(nums):
                    return False
                if nums[nums_idx]==n:
                    nums_idx+=1
                else:
                    return False
            n=0
    if check_all:
        if n>0:
            if nums_idx>=len(nums):
                return False
            if nums[nums_idx]==n:
                nums_idx+=1
            else:
                return False
        return nums_idx>=len(nums)
    else:
        return True

In [ ]:
# 2023 day 12 part 0
# tests for check_nums

assert check_nums('.###....#', [3, 2, 1], True)==False
assert check_nums('.###....#', [3, 2, 1], False)==True
assert check_nums('.###....##.#', [3, 2, 1], True)==True
assert check_nums('.###....##.#.#.#.#', [3, 2, 1], False)==False

In [ ]:
# 2023 day 12 part 1
# mv ~/Downloads/input* data_src/2023-day-12-input.txt
# big input file looks like: 1000 short lines
# idea: part 1 parse each line as a string and list of numbers, then depth first search on each char in the string

sample2='''
???.### 1,1,3
.??..??...?##. 1,1,3
?#?#?#?#?#?#?#? 1,3,1,6
????.#...#... 4,1,1
????.######..#####. 1,6,5
?###???????? 3,2,1
'''

def check_arran(s, line, nums):
    if len(s)>=len(line):
        if not check_nums(s, nums, True):
            return 0
        return 1
    else:
        if not check_nums(s, nums, False):
            return 0
        c=line[len(s)]
        if c=='?':
            total=0
            total+=check_arran(s+'.', line, nums)
            total+=check_arran(s+'#', line, nums)
            return total
        else:
            assert c=='.' or c=='#'
            total=check_arran(s+c, line, nums)
            return total

def count_arran(line, nums):
    '''count possible arrangements'''
    cnt=check_arran('', line, nums)
    #print(f'count_arran: {line=}, {cnt=}')
    return cnt

sample1=open('data_src/2023-day-12-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
data=[]
for line in lines:
    tup=line.split()
    assert len(tup)==2
    nums=[ int(n) for n in tup[1].split(',') ]
    data.append( (tup[0], nums) )

In [ ]:
# 2023 day 12 part 1 run

total=0
for tup in data:
    total+=count_arran(*tup)
print(f'part 1: {total=}')

# part 1: 7090

In [ ]:
# 2023 day 12 part 2
# idea: DFS as implemented above takes too long, 
# top 3 lines of actual input:
#  .#?#???????.????# 1,2,3,2,1  # cnt=49762
#  ?????????? 1,1,4
#  ????.??.??.??? 1,2
# unfolded already take way too long this way, so a new approach:
# - remove . from start and end, replace .. by ., split on .
# - now DFS on nums, maintaining a remaining list of strings and a multiplication factor
# - if only ? remain in a string, calculate a multiplication factor for ways to put n next nums in those ?
# - cutoff if remaining strings is less than nums+min.separators
# this still takes quite long, so there has to be a faster way but for now this is ok-ish

sample3='''
?###???????? 3,2,1
'''

sample4='''
.#?#???????.????# 1,2,3,2,1
?????????? 1,1,4
????.??.??.??? 1,2
'''

sample5='''
??????#??????.? 2,4,1,1,1
'''

def unfold_tup(line, nums):
    s=line
    nums2=list(nums)
    for _ in range(4):
        s+='?'+line
        nums2.extend(nums)
    while s.startswith('.'):
        s=s[1:]
    while s.endswith('.'):
        s=s[:-1]
    while '..' in s:
        s=s.replace('..', '.')
    return s.split('.'), nums2

#unfold_tup('.#', [1])
#unfold_tup('???.###', [1,1,3])

def count_arran2(substr, si, atsubstart, nums, ni, mult, tslen, tnlen):
    """
    count possible arrangements /fits of nums[ni:] into substr[si:]

    Args:
        substr (list of strings): strings to fit the numbers in (each only contains ? and #)
        si (int): index in substr to current string
        atsubstart (bool): true if at the start of substr[si], false otherwise
        nums (list of int): list of numbers to fit into substr (each a length of damaged springs)
        ni (int): index in nums to current number to match
        mult (int): multiplier of number of arrangements to return
        tslen (int): total length of strings in substr, plus separators, still available
        tnlen (int): minimum total length of nums that still need to be fit

    Returns:
        int: number of different ways to fit
    """
    #print(f'count_arran2: {substr=}, {si=}, {atsubstart=}, {nums=}, {ni=}, {mult=}')
    #assert tslen==sum([len(s) for s in substrs[si:]])+len(substrs[si:])-1
    #assert tnlen==sum(nums[ni:])+len(nums[ni:])-1
    while si<len(substr) and substr[si]=='': # skip any empty substr
        tslen-=1
        si+=1
        atsubstart=True
    if ni>=len(nums): # done, are we good? (only ? remaining)
        for s in substr[si:]:
            if '#' in s:
                return 0        
        return mult
    if si>=len(substr): # all substr processed but nums remaining, not good
        return 0
    if tnlen>tslen: # remaining nums cannot fit
        return 0
    s=substr[si]
    if '#' in s: # match a separator of 1-n (0-n at start), then the next num
        cnt=0
        if atsubstart and len(s)>=nums[ni]:
            substr[si]=s[nums[ni]:]
            cnt+=count_arran2(substr, si, False, nums, ni+1, mult,
                              tslen-nums[ni], tnlen-nums[ni]-1)
        for i in range(0, len(s)-nums[ni]): # e.g. s is 5 chars, nums[ni] is 2 so first 3 can be sep
            if s[i]=='?':
                substr[si]=s[i+1+nums[ni]:]
                cnt+=count_arran2(substr, si, False, nums, ni+1, mult,
                                  tslen-nums[ni]-i-1, tnlen-nums[ni]-1)
            else:
                break
        substr[si]=s
        return cnt
    else: # only ?, update mult, but mind atsubstart, for 0-n nums how many ways?
        if not atsubstart: # have to start w/ a separator
            s=s[1:]
            tslen-=1
            atsubstart=True
        cnt=0
        cnt+=count_arran2(substr, si+1, True, nums, ni, mult,  # skip this substr by putting only sep in it
                          tslen-len(s)-1, tnlen)
        for nnums in range(1, len(nums)-ni+1): # calculate ways for next nnums nums to fit in
            ntot=sum(nums[ni:ni+nnums]) # characters covered by the nums
            nrem=len(s)-ntot-(nnums-1) # remaining after nums and mandatory separators
            if nrem<0: # doesn't fit, can stop here
                break
            # how many ways can we divide nrem marbles over nnums+1 bin?
            # formulated as a stars-and-bars problem:
            # (see https://brilliant.org/wiki/identical-objects-into-distinct-bins/)
            # we have to place nnums bars in nrem+nnums positions, so nrem+nnums over nnums
            ways=math.comb(nrem+nnums, nnums)
            assert ways>0
            cnt+=count_arran2(substr, si+1, True, nums, ni+nnums, mult*ways,
                              tslen-len(s)-1, tnlen-ntot-nnums)
        substr[si]=s
        return cnt

sample1=open('data_src/2023-day-12-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]

total=0
for line in lines:
    tup=line.split()
    assert len(tup)==2
    nums=[ int(n) for n in tup[1].split(',') ]
    substrs,nums=unfold_tup(tup[0], nums)
    cnt=count_arran2(substrs, 0, True, nums, 0, 1,
                     sum([len(s) for s in substrs])+len(substrs)-1, sum(nums)+len(nums)-1)
    print(f'count_arran2: {substrs=}, {nums=}, {cnt=}')
    total+=cnt
print(f'part 2: {total=}')

# part 2: 6792010726878 (run time: 105 minutes!!)

In [ ]:
# 2023 day 11 part 1
# mv ~/Downloads/input* data_src/2023-day-11-input.txt
# big input file looks like: big map
# idea: part 1 parse as list of lines, then expand in the lines,
# then convert to a list of coordinates and calculate manhattan distance

sample2='''
...#......
.......#..
#.........
..........
......#...
.#........
.........#
..........
.......#..
#...#.....
'''

sample1=open('data_src/2023-day-11-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]

# first expand columns
x=0
while x<len(lines[0]):
    foundgal=False
    for y in range(len(lines)):
        if lines[y][x]=='#':
            foundgal=True
    if not foundgal:
        for y in range(len(lines)):
            lines[y]=lines[y][:x]+' '+lines[y][x:]
        x+=2
    else:
        x+=1
# expand rows
y=0
while y<len(lines):
    foundgal='#' in lines[y]
    if not foundgal:
        lines.insert(y, '')
        y+=2
    else:
        y+=1

#print('expanded:')
#for line in lines:
#    print(line)

def manh_dist(a, b):
    '''manhattan distance between two positions, each a tuple of x,y'''
    dist=abs(a[0]-b[0])+abs(a[1]-b[1])
    return dist

def total_dist(data):
    '''based on list of x,y positions calculate total manhattan distance between
    each pair'''
    total=0
    for pair in itertools.combinations(data, 2):
        total+=manh_dist(*pair)
    return total

data=[] # (x,y) of galaxy
for y,row in enumerate(lines):
    for x,c in enumerate(row):
        if c=='#':
            data.append( (x,y) )
total=total_dist(data)
print(f'part 1: {total=}')

# part 1: 10033566

In [ ]:
# 2023 day 11 part 2
# idea: based on lists of empty row and column numbers, shift/expand each galaxy
# while converting from the map to list of coordinates, then calculate total distance 
# as before

def empty_rows(lines):
    '''return list of empty row numbers'''
    return [ y for y,row in enumerate(lines) if '#' not in row ]

def empty_cols(lines):
    '''return list of empty column numbers'''
    return [ x for x in range(len(lines[0])) if '#' not in { lines[y][x] for y in range(len(lines)) } ]

sample1=open('data_src/2023-day-11-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
erows=empty_rows(lines)
ecols=empty_cols(lines)
print(f'erows: {erows}')
print(f'ecols: {ecols}')

data=[] # (x,y) of galaxy, expanded
fact=1000000
for y,row in enumerate(lines):
    for x,c in enumerate(row):
        if c=='#':
            newx=x
            for n in ecols:
                if n<x:
                    newx+=fact-1
            newy=y
            for n in erows:
                if n<y:
                    newy+=fact-1
            data.append( (newx,newy) )
total=total_dist(data)
print(f'part 2: {total=}')

In [ ]:
# TEMPLATE
# 2023 day 11
# start_ts=RUN FIRST CELL TO GET TIME CODE BEFORE OPENING THE ASSIGNMENT
# mv ~/Downloads/input* data_src/2023-day-11-input.txt
# big input file looks like: 
# idea: part 1 parse ..., then ...

sample2='''

'''

sample1=open('data_src/2023-day-11-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
groups=zio.get_line_groups(sample1.splitlines(), nostrip=False)
data=[ int(s) for s in lines[0].split(',') ]
data=[ s.split() for s in lines ]
data=[ [cmd, int(num), 0] for cmd, num in data ]
data=[ result.group(1, 2, 3, 4, 5, 6, 7) for s in lines if (result:= re.match(r'(\w+)\s*x=([\d\-]+)\.\.([\d\-]+),y=([\d\-]+)\.\.([\d\-]+),z=([\d\-]+)\.\.([\d\-]+)', s)) ]
data=[ (row[0], int(row[1]), int(row[2]), int(row[3]), int(row[4]), int(row[5]), int(row[6]) ) for row in data ]
# template, remove what's not needed